In [108]:
# This is a project type notebook, it needs huge computational power and time for the full IMDB Dataset

In [109]:
!pip install nltk

In [110]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\saims\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [111]:
import pandas as pd

In [112]:
# importing the dataframe

In [113]:
# df = pd.read_csv('IMDB Dataset.csv')

# taking only first 1000 rows
df = pd.read_csv('IMDB Dataset.csv').head(1000)

In [114]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
995,Nothing is sacred. Just ask Ernie Fosselius. T...,positive
996,I hated it. I hate self-aware pretentious inan...,negative
997,I usually try to be professional and construct...,negative
998,If you like me is going to see this in a film ...,negative


In [115]:
# EDA

In [116]:
df.shape

(1000, 2)

In [117]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [118]:
df.describe()

,review,sentiment
count,1000,1000
unique,1000,2
top,One of the other reviewers has mentioned that ...,positive
freq,1,501


In [119]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     1000 non-null   object
 1   sentiment  1000 non-null   object
dtypes: object(2)
memory usage: 15.8+ KB


In [120]:
df['sentiment'].unique()

array(['positive', 'negative'], dtype=object)

In [121]:
df['sentiment'].value_counts()

positive    501
negative    499
Name: sentiment, dtype: int64

In [122]:
# apply label encoder

In [123]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [124]:
df['sentiment'] = le.fit_transform(df['sentiment'])

In [125]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [126]:
X = df['review']
y = df['sentiment']

In [127]:
X

0      One of the other reviewers has mentioned that ...
1      A wonderful little production. <br /><br />The...
2      I thought this was a wonderful way to spend ti...
3      Basically there's a family where a little boy ...
4      Petter Mattei's "Love in the Time of Money" is...
                             ...                        
995    Nothing is sacred. Just ask Ernie Fosselius. T...
996    I hated it. I hate self-aware pretentious inan...
997    I usually try to be professional and construct...
998    If you like me is going to see this in a film ...
999    This is like a zoology textbook, given that it...
Name: review, Length: 1000, dtype: object

In [128]:
y

0      1
1      1
2      1
3      0
4      1
      ..
995    1
996    0
997    0
998    0
999    0
Name: sentiment, Length: 1000, dtype: int32

In [129]:
# removing all special and numeric character from the data and also remove stopwords and apply stemming

In [130]:
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re

In [131]:
ps = PorterStemmer()
corpus = []

for i in range(len(X)):
    print(i, end=', ')
    review = re.sub("[^a-zA-Z]"," ",X[i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if word not in set(stopwords.words("english"))]
    review = " ".join(review)
    corpus.append(review)

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 

In [132]:
print(corpus)

['one review mention watch oz episod hook right exactli happen br br first thing struck oz brutal unflinch scene violenc set right word go trust show faint heart timid show pull punch regard drug sex violenc hardcor classic use word br br call oz nicknam given oswald maximum secur state penitentari focus mainli emerald citi experiment section prison cell glass front face inward privaci high agenda em citi home mani aryan muslim gangsta latino christian italian irish scuffl death stare dodgi deal shadi agreement never far away br br would say main appeal show due fact goe show dare forget pretti pictur paint mainstream audienc forget charm forget romanc oz mess around first episod ever saw struck nasti surreal say readi watch develop tast oz got accustom high level graphic violenc violenc injustic crook guard sold nickel inmat kill order get away well manner middl class inmat turn prison bitch due lack street skill prison experi watch oz may becom comfort uncomfort view that get touch d

In [133]:
from sklearn.feature_extraction.text import TfidfVectorizer
# cv = TfidfVectorizer(max_features=5000)
cv = TfidfVectorizer(max_features=100)
X = cv.fit_transform(corpus).toarray()

In [134]:
X

array([[0.        , 0.        , 0.        , ..., 0.10489191, 0.        ,
        0.        ],
       [0.        , 0.15819564, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.1685287 ,
        0.20273143],
       ...,
       [0.11972567, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.10559737, 0.        , 0.        , ..., 0.09487176, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [135]:
X.shape

(1000, 100)

In [136]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=81)

In [137]:
from sklearn.svm import SVC
model = SVC(C=1.0,
           kernel='linear',
           random_state=81)

In [138]:
model.fit(X_train, y_train)

SVC(kernel='linear', random_state=81)

In [139]:
model.score(X_test, y_test)

0.665

In [140]:
y_pred = model.predict(X_test)

In [141]:
# save and load the model
import pickle

In [142]:
pickle.dump(cv, open('movie_review_cv.pkl', 'wb'))
pickle.dump(model, open('movie_review_model.pkl', 'wb'))

In [143]:
save_cv = pickle.load(open('movie_review_cv.pkl', 'rb'))
save_model = pickle.load( open('movie_review_model.pkl', 'rb'))

In [144]:
# function for testing

def test_model(sentence):
    sen = save_cv.transform([sentence]).toarray()
    res = save_model.predict(sen)[0]
#     print(res)
    if res == 1:
        print('positive')
    else:
        print('negative')

In [145]:
sentence = 'i love amber heard'
test_model(sentence)

positive


In [146]:
sentence = 'i hate gigi hadid'
test_model(sentence)

negative


In [147]:
print(set(stopwords.words("english")))

{'ours', 'whom', 'she', 'them', 'couldn', 'having', 'than', 'which', 'where', 'off', 'him', "that'll", 'here', 'haven', 'through', 's', 'a', 'above', 'out', "mightn't", 'ma', 'aren', 'only', 'himself', 'o', 'there', 'y', 'our', 'own', 'what', 'these', 'for', 'over', 'if', "weren't", 'until', 'at', 'hers', 'm', "hasn't", 'shouldn', 'from', 'yourself', 'before', 'to', 'myself', 'few', 'being', 'wasn', 'am', 'my', 'why', 'same', 'does', 'any', 'very', 'weren', 'isn', 'ain', 'you', 'how', 'once', 'that', 'be', 'its', 'needn', 'shan', 'was', 'they', 'had', 'such', 'now', 'in', "shan't", 'most', 'an', 'doing', 'about', "she's", 've', 'i', 'herself', 'the', 'or', "hadn't", 'further', 'during', 'with', 'below', "you're", 'itself', 'me', 'theirs', 'so', 'can', 'll', "haven't", 'up', 'd', 'as', 'hadn', 'it', 'too', 're', 'ourselves', 'he', 'do', 'is', "you'll", 'have', 'of', "don't", 'has', "you'd", 'because', 'just', 'wouldn', 'yours', "aren't", 'then', 'nor', "didn't", 'under', "doesn't", 'aga

In [148]:
from sklearn.metrics import classification_report
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.70      0.64      0.67       105
           1       0.63      0.69      0.66        95

    accuracy                           0.67       200
   macro avg       0.67      0.67      0.66       200
weighted avg       0.67      0.67      0.67       200

